In [1]:
# TODO: Find out why we can't deamonize the NEST server...
# For the moment we need to use Ray to deamonize it, or start it on a terminal.
# ! nest-server start -o -p 52425

import os
import ray

ray.init()  # _temp_dir="/home/docker/tmp/ray", object_store_memory=100000000


@ray.remote
def start_deamonize_nest_server():
    os.system('! nest-server start -o -p 52425')

# @ray.remote
# def nest_server_status():
#     os.system('! nest-server status')
    
# @ray.remote
# def nest_server_log():
#     os.system('! nest-server log')



2023-09-20 11:57:22,623	INFO worker.py:1642 -- Started a local Ray instance.


In [2]:
start_deamonize_nest_server.remote()
# nest_server_status.remote()
# nest_server_log.remote()

ObjectRef(c8ef45ccd0112571ffffffffffffffffffffffff0100000001000000)

(start_deamonize_nest_server pid=1585) NEST Server is now running at http://127.0.0.1:52425.
(start_deamonize_nest_server pid=1585) Use CTRL + C to stop this service.
(start_deamonize_nest_server pid=1585) -------------------------------------------------
(start_deamonize_nest_server pid=1585) 
(start_deamonize_nest_server pid=1585)               -- N E S T --
(start_deamonize_nest_server pid=1585)   Copyright (C) 2004 The NEST Initiative
(start_deamonize_nest_server pid=1585) 
(start_deamonize_nest_server pid=1585)  Version: 3.5
(start_deamonize_nest_server pid=1585)  Built: Sep 19 2023 15:56:48
(start_deamonize_nest_server pid=1585) 
(start_deamonize_nest_server pid=1585)  This program is provided AS IS and comes with
(start_deamonize_nest_server pid=1585)  NO WARRANTY. See the file LICENSE for details.
(start_deamonize_nest_server pid=1585) 
(start_deamonize_nest_server pid=1585)  Problems or suggestions?
(start_deamonize_nest_server pid=1585)    Visit https://www.nest-simulator.org

In [3]:
from nest_client import NESTClient

print('Running client examples using NEST via NEST Server')

# Load NEST Server client
nestsc = NESTClient(host="127.0.0.1", port=52425)

#
# Use NEST Server API
#
print('\n')
print('Execute script code with NEST Server API')
print('-' * 20)

# Reset kernel
nestsc.ResetKernel()
nestsc.GetKernelStatus("biological_time")

# Create nodes
pg = nestsc.Create("poisson_generator", params={"rate": 6500.})
neurons = nestsc.Create("iaf_psc_alpha", 100)
sr = nestsc.Create("spike_recorder")

# Connect nodes
nestsc.Connect(pg, neurons, syn_spec={'weight': 10.})
nestsc.Connect(neurons[::10], sr)

# Simulate
nestsc.Simulate(1000.0)

# Get events
n_events = nestsc.GetStatus(sr, 'n_events')[0]
print('Number of events:', n_events)


#
# Use NEST Server exec
#
print('\n')
print('Execute script code from file')
print('-' * 20)

n_events = nestsc.from_file('/home/docker/packages/nest-client/examples/NESTClient_script.py', 'n_events')['data']
print('Number of events:', n_events)

Running client examples using NEST via NEST Server


Execute script code with NEST Server API
--------------------
(start_deamonize_nest_server pid=1585) 
(start_deamonize_nest_server pid=1585) ========================================
(start_deamonize_nest_server pid=1585) 
(start_deamonize_nest_server pid=1585) ==> MASTER 0/1695211057.3136008 (route_api_call): call=ResetKernel, args=[], kwargs={}
(start_deamonize_nest_server pid=1585) ==> MASTER 0/1695211057.3136392 (ResetKernel): local call, args=[], kwargs={}
(start_deamonize_nest_server pid=1585) 
(start_deamonize_nest_server pid=1585) ========================================
(start_deamonize_nest_server pid=1585) 
(start_deamonize_nest_server pid=1585) ==> MASTER 0/1695211057.3157861 (route_api_call): call=GetKernelStatus, args=['biological_time'], kwargs={}
(start_deamonize_nest_server pid=1585) ==> MASTER 0/1695211057.3158243 (GetKernelStatus): local call, args=['biological_time'], kwargs={}
(start_deamonize_nest_server pid=1585

(start_deamonize_nest_server pid=1585) 
(start_deamonize_nest_server pid=1585) Sep 20 11:57:37 NodeManager::prepare_nodes [Info]: 
(start_deamonize_nest_server pid=1585)     Preparing 102 nodes for simulation.
(start_deamonize_nest_server pid=1585) 
(start_deamonize_nest_server pid=1585) Sep 20 11:57:37 SimulationManager::start_updating_ [Info]: 
(start_deamonize_nest_server pid=1585)     Number of local nodes: 102
(start_deamonize_nest_server pid=1585)     Simulation time (ms): 1000
(start_deamonize_nest_server pid=1585)     Number of OpenMP threads: 1
(start_deamonize_nest_server pid=1585)     Number of MPI processes: 1
(start_deamonize_nest_server pid=1585) 
(start_deamonize_nest_server pid=1585) Sep 20 11:57:37 SimulationManager::run [Info]: 
(start_deamonize_nest_server pid=1585)     Simulation finished.


In [4]:
from tvb_multiscale.tvb_nest.nest_models.server_client.nest_server_client import NESTServerClient 

# Load NEST Server client
nest = NESTServerClient(host='localhost', port=52425)

nest.GetKernelStatus("biological_time")

1000.0

(start_deamonize_nest_server pid=1585) 
(start_deamonize_nest_server pid=1585) ========================================
(start_deamonize_nest_server pid=1585) 
(start_deamonize_nest_server pid=1585) ==> MASTER 0/1695211064.0701878 (route_api_call): call=GetKernelStatus, args=['biological_time'], kwargs={}
(start_deamonize_nest_server pid=1585) ==> MASTER 0/1695211064.0702314 (GetKernelStatus): local call, args=['biological_time'], kwargs={}


In [5]:
! curl localhost:52425

/usr/bin/sh: 1: curl: not found


In [6]:
import requests

requests.get('http://localhost:52425').json()
requests.get('http://127.0.0.1:52425/api/GetKernelStatus', json=["biological_time"]).json()

1000.0

(start_deamonize_nest_server pid=1585) 
(start_deamonize_nest_server pid=1585) ========================================
(start_deamonize_nest_server pid=1585) 
(start_deamonize_nest_server pid=1585) ==> MASTER 0/1695211067.1835012 (route_api_call): call=GetKernelStatus, args=['biological_time'], kwargs={}
(start_deamonize_nest_server pid=1585) ==> MASTER 0/1695211067.1837599 (GetKernelStatus): local call, args=['biological_time'], kwargs={}


In [7]:
# ! /home/docker/env/neurosci/nest_build/bin/nest-server stop
! nest-server stop

NEST Server running at http://127.0.0.1:52425 has stopped.


In [8]:
ray.shutdown()